In [1]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
df = pd.read_csv('data1.csv')
df.head()

,temp,gender,height(cm),weight(kg),fit,c_result
0,28,여자,162,47,큼,M1
1,28,여자,170,55,적당함,S1
2,28,여자,162,55,큼,M1
3,28,여자,171,75,큼,L1
4,28,여자,167,50,큼,M1


In [4]:
df_encoded = pd.get_dummies(df,columns=['gender','fit'])
df_encoded.head()

,temp,height(cm),weight(kg),c_result,gender_남자,gender_여자,fit_작음,fit_적당함,fit_큼
0,28,162,47,M1,0,1,0,0,1
1,28,170,55,S1,0,1,0,1,0
2,28,162,55,M1,0,1,0,0,1
3,28,171,75,L1,0,1,0,0,1
4,28,167,50,M1,0,1,0,0,1


In [5]:
from sklearn.preprocessing import StandardScaler


cols_to_scale = ['temp', 'height(cm)', 'weight(kg)']
scaler = StandardScaler()

df_encoded[cols_to_scale] = scaler.fit_transform(df_encoded[cols_to_scale])

df_encoded.head()

,temp,height(cm),weight(kg),c_result,gender_남자,gender_여자,fit_작음,fit_적당함,fit_큼
0,1.609488,-1.202216,-1.567394,M1,0,1,0,0,1
1,1.609488,-0.189824,-0.910017,S1,0,1,0,1,0
2,1.609488,-1.202216,-0.910017,M1,0,1,0,0,1
3,1.609488,-0.063275,0.733426,L1,0,1,0,0,1
4,1.609488,-0.569471,-1.320877,M1,0,1,0,0,1


In [6]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [7]:
X = df_encoded.drop('c_result', axis=1)
y = df_encoded['c_result']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [11]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(n_estimators=400, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92], got ['L1' 'L10' 'L11' 'L14' 'L15' 'L17' 'L19' 'L2' 'L20' 'L22' 'L23' 'L24'
 'L26' 'L27' 'L29' 'L31' 'L33' 'L35' 'L36' 'L37' 'L38' 'L39' 'L5' 'L6'
 'L8' 'L9' 'M1' 'M10' 'M11' 'M14' 'M15' 'M17' 'M19' 'M2' 'M20' 'M22' 'M23'
 'M24' 'M26' 'M27' 'M29' 'M3' 'M31' 'M33' 'M35' 'M36' 'M38' 'M39' 'M5'
 'M6' 'M8' 'M9' 'S1' 'S14' 'S15' 'S17' 'S19' 'S20' 'S22' 'S24' 'S26' 'S27'
 'S3' 'S31' 'S33' 'S35' 'S36' 'S38' 'S39' 'S5' 'S6' 'XL1' 'XL10' 'XL14'
 'XL15' 'XL17' 'XL19' 'XL2' 'XL20' 'XL24' 'XL26' 'XL27' 'XL31' 'XL35'
 'XL36' 'XL39' 'XL6' 'XL8' 'XL9' 'XS1' 'XXL1' 'XXL24' 'XXL31']

## 일단 오류나서 gradient로 해보겠음

In [12]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

In [13]:
gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
gb_model.fit(X_train, y_train)

GradientBoostingClassifier(random_state=42)

In [14]:
y_pred = gb_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6739130434782609


## 오버샘플링

In [15]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42, k_neighbors=3)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [18]:
gb_model2 = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
gb_model2.fit(X_train_resampled, y_train_resampled)

GradientBoostingClassifier(random_state=42)

In [17]:
y_pred = gb_model2.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6739130434782609
